In [20]:
#!pip install pynput

In [ ]:
# creating the classes
from pynput import keyboard as kb

class HauntedMansion:
    def __init__(self):
        self.rooms = {}
        self.doors = {}

    def add_room(self, *rooms):
        for room in rooms:
            self.rooms[room.name] = room

    def add_door(self, *doors):
        for door in doors:
            if door.room1.name in self.rooms and door.room2.name in self.rooms:
                self.doors[door.name] = door
                door.room1.add_door(door)
                door.room2.add_door(door)

class Room:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.objects = {}
        self.doors = []

    def add_door(self, door):
         if door not in self.doors:
             self.doors.append(door)

    def add_object(self, *objs):
        for obj in objs:
            self.objects[obj.name] = obj

    def search(self):
        if self.objects:
            print("You found:")
            for obj in self.objects.values():
                print(f"- {obj.name}: {obj.description}")
        else:
            print("There's nothing interesting here.")
        if self.doors:
             print(f"You also find that there are {len(self.doors)} doors:")
             for door in self.doors:
                print(f"- {door.name}")
        else:
             print("There are no doors here.")

class Object:
    def __init__(self, name, description, item=None):
        self.name = name
        self.description = description
        self.item = item

    def interact(self, player):
        if self.item:
            print(f"You found a {self.item.name} inside the {self.name}!")
            player.inventory.append(self.item)
            self.item = None  # The item is taken
        else:
            print(f"The {self.name} is empty.")

class Chest(Object):
    def __init__(self, name, description, item = None, is_locked=True, correct_pin = None):
        super().__init__(name, description, item)
        self.is_locked = is_locked
        self.correct_pin = correct_pin 
    
    def interact(self, player):
        if self.is_locked:
            print(f"The {self.name} is locked, try to open it.! Pin-code is {self.correct_pin}")
            while True:
                pin = input()
                if pin.lower() == "cancel":
                    print("You decided not to try the PIN code.")
                    return  
                try:
                    if len(pin) != 4 or not pin.isdigit():  # check length and if cod contains digits
                        raise ValueError
                    if int(pin) == self.correct_pin:  # check if the pin is correct
                        self.is_locked = False
                        print(f"You unlocked the {self.name}!")
                        super().interact(player)  # open the chest 
                        return  
                    else:
                        print("Incorrect PIN. Try again or type 'cancel':")
                except ValueError:
                    print("Invalid input. Please enter a 4-digit PIN or type 'cancel':")

        else: 
            print("It looks like someone already opened it. There's nothing interesting here.")
    

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Door:
    def __init__(self, name, room1, room2, key_required = None):
        self.name = name
        self.room1 = room1
        self.room2 = room2
        self.key_required = key_required

    def get_other_room(self, current_room):
        return self.room2 if current_room == self.room1 else self.room1

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []
        self.current_action = None

    def game_loop(self):
        instructions = "Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit."
        further_instructions = "To move through a door or interact with an object, please press the corresponding number of your choice."
        exit_moving = "Press 'Esc' to quit the move."
        exit_interacting = "Press 'Esc' to quit the interaction."

        print(f"\nYou are in {self.current_room.name}. {self.current_room.description}")
        print(instructions + " " + further_instructions)
        print("Use 'H' to display instructions again at any point.")

        self.running = True

        def on_press(key):
            try:
                if key.char == "h":  # Show instructions
                    print("\n--- Instructions ---")
                    if self.current_action == "moving":
                        print(further_instructions + "\n" + exit_moving)
                    elif self.current_action == "interacting":
                        print(further_instructions + "\n" + exit_interacting)
                    else:
                        print(instructions)
                    print("--------------------")
                elif key.char == "w":
                    self.move()
                elif key.char == "q":
                    self.search_room()
                elif key.char == "e":
                    self.interact_with_object()
            except AttributeError:
                # Special keys (like Esc) don't have `.char`
                if key == kb.Key.esc:
                    print("Exiting game...")
                    self.running = False  # Stop the game loop
                    return False  # Stops listener

        # Start listening for keypresses
        with kb.Listener(on_press=on_press) as listener:
            while self.running:
                pass  # Keep game running until self.running is False

        print("Game loop ended.")

    def move(self):
        if self.current_action:
            return
        
        self.current_action = "moving"
        doors = self.current_room.doors
        if not doors:
            print("There are no doors here. How did you even get here?")
            self.current_action = None
            return
        
        print("\nWhere do you want to move?")
        for index, door in enumerate(doors, start=1):
            print(f"{index} - {door.name}")

        while True:
            event = kb.read_event()
            if event.event_type == "down":
                if event.name.isdigit():
                    index = int(event.name)
                    if 1 <= index <= len(doors):
                        selected_door = doors[index - 1]
                        other_room = selected_door.get_other_room(self.current_room)
                        if selected_door.key_required and selected_door.key_required not in self.inventory:
                            print(f"You need a {selected_door.key_required.name} to go there.")
                        else:
                            self.current_room = other_room
                            print(f"You moved to {other_room.name}. {other_room.description}")
                            break
                elif event.name == "esc":
                    print("Exiting movement... What do you want to do now?")
                    break

        self.current_action = None
    
    def search_room(self):
        self.current_room.search()

    def interact_with_object(self):
        if self.current_action:
            return
        
        self.current_action = "interacting"
        objects = self.current_room.objects
        if not objects:
            print("There's nothing to interact with here.")
            self.current_action = None
            return

        print("\nWhat do you want to interact with?")
        for index, obj in enumerate(objects.values(), start=1):
            print(f"{index} - {obj.name}")
        while True:
            event = kb.read_event()
            if event.event_type == "down":
                if event.name.isdigit():
                    index = int(event.name)
                    if 1 <= index <= len(objects):
                        obj = list(objects.values())[index - 1]
                        obj.interact(self)
                        break
                elif event.name == "esc":
                    print("Exiting interaction... What do you want to do now?")
                    break
        self.current_action = None

In [ ]:
# creating & setting the game

# Create a house
house = HauntedMansion()

# Create rooms
# Grond floor
kitchen = Room("Kitchen", "A warm kitchen with a smell of fresh bread.")
hallway = Room("Hallway", "An empty hallway.")
dining_room = Room("Dining Room", "A nice room to have a meal.")
outside = Room("Outside", "A breeze of fresh air and singing birds. Congratulations, you've made it to the outside and won the game!")
living_room = Room("Living Room", "A huge living room. You hear some songs from the radio.")
wc = Room("WC", "A small room with a toilet and a sink.")
# first floor
bedroom = Room("Bedroom", "A peaceful bedroom with a soft bed.")
little_guest_room = Room("Little guest Room", "A little room for guests.")
big_guest_room = Room("Big guest Room", "A big room for guests.")
hallway_first_floor = Room("Hallway first floor", "An empty hallway on the first floor.")
bath = Room("Bath", "A bathroom with a bathtub and a shower.")
bathroom = Room("Bathroom", "A cold bathroom with a toilet and a sink.")

# keys
silver_key = Item("Silver key", "Opens the door between the dining room and the kitchen")
golden_key = Item("Golden key", "Opens the door between the hallway and the living room")
bronze_key = Item("Bronze key", "Opens the door between the first floor hallway and the guest room")
platin_key = Item("Platin key", "Opens the door between the first floor hallway and the bedroom")
master_key = Item("Master key", "Opens the big door")



# Add rooms to the house
house.add_room(kitchen, hallway, bedroom, dining_room, outside, living_room, wc, 
               little_guest_room, big_guest_room, hallway_first_floor, bath, bathroom)


# Create doors
# Grond floor
silver_door = Door("Silver door", dining_room, kitchen, silver_key)
white_door = Door("White door", dining_room, hallway)
gold_door = Door("Gold door", hallway, living_room, golden_key)
big_door = Door("Big door", hallway, outside, master_key)
green_door = Door("Green door", hallway, wc)
stairs = Door("Stairs", hallway, hallway_first_floor)
# first floor
bronze_door = Door("Bronze door", hallway_first_floor, little_guest_room, bronze_key)
platin_door = Door("Platin door", hallway_first_floor, bedroom, platin_key)
black_door = Door("Black door", hallway_first_floor, bath)
red_door = Door("Red door", hallway_first_floor, big_guest_room)
yellow_door = Door("Yellow door", bedroom, bathroom)




# Connect rooms with doors
house.add_door(silver_door, white_door, gold_door, big_door, green_door, stairs, 
               bronze_door, platin_door, black_door, red_door, yellow_door)



# Create a player and set their starting room
player = Player()
player.current_room = dining_room

# Create a drawer that contains a key
# ground floor
chest = Chest("Chest", "A mysterious chest.", golden_key, correct_pin = 1920)
cupbourd = Object("Cupboard", "A brown wooden cupboard.")
drawer = Object("Drawer", "An old wooden drawer.")
cabinet = Object("Cabinet", "A huge black cabinet.")
radio = Object("Radio", "An ancient radio.")

# first floor
mirror = Object("Mirror", "A big durty mirror with a golden frame.")
little_drawer = Object("Little drawer", "A little drawer.")
desk = Object("Desk", "A wooden desk.")
bed = Object("Bed", "A messy bed. You woke up here!")
king_bed = Object("King bed", "A huge bed.")
safe = Object("Safe", "A safe with a digital lock.")


# Add obejcts to rooms
dining_room.add_object(drawer)
kitchen.add_object(chest, cupbourd)
bedroom.add_object(bed)
little_guest_room.add_object(little_drawer, mirror)
big_guest_room.add_object(bed, desk)
bedroom.add_object(king_bed, safe)
living_room.add_object(radio, cabinet)


In [ ]:
player.game_loop()


You are in Dining Room. A nice room to have a meal.
Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit. To move through a door or interact with an object, please press the corresponding number of your choice.
Use 'H' to display instructions again at any point.



--- Instructions ---
Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit.
--------------------

What do you want to interact with?
1 - Drawer


KeyboardInterrupt: 

You found:
- Drawer: An old wooden drawer.
You also find that there are 2 doors:
- Silver door
- White door

--- Instructions ---
To move through a door or interact with an object, please press the corresponding number of your choice.
Press 'Esc' to quit the interaction.
--------------------
You found:
- Drawer: An old wooden drawer.
You also find that there are 2 doors:
- Silver door
- White door
You found:
- Drawer: An old wooden drawer.
You also find that there are 2 doors:
- Silver door
- White door

--- Instructions ---
To move through a door or interact with an object, please press the corresponding number of your choice.
Press 'Esc' to quit the interaction.
--------------------

--- Instructions ---
To move through a door or interact with an object, please press the corresponding number of your choice.
Press 'Esc' to quit the interaction.
--------------------

--- Instructions ---
To move through a door or interact with an object, please press the corresponding number of your ch